<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Personality%20Enabled%20MF/Personality%20prediction%20from%20text%20using%20bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# source to be clustered with dbscan/kmeans
# target instances to be merged with clusters from source
# joint training and embedding on individual subsets
# prediction on target sets

In [ ]:
# importing the packages
import numpy as np
import scipy.sparse as sp
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
import scipy.spatial as spt
import statistics
import math
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# data collection

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})
arr = tr[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating', 'processed_text']]

In [ ]:
# amazon review dataset magazine json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile)
#az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})
az.head()

In [ ]:
tr1 = arr[0:90]
tr2 = arr[90:100]
source_texts = np.array(tr1.processed_text)
target_texts = np.array(tr2.processed_text)

# Clustering of text

In [ ]:
# clustering of sorce domain reviews 

def cluster_text(X):
    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    Sum_of_squared_distances = []
    K = range(2,20)
    for k in K:
       km = KMeans(n_clusters=k, max_iter=200, n_init=5)
       km = km.fit(X)
       Sum_of_squared_distances.append(km.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    print('How many clusters do you want to use?')
    no_of_clusters = int(input())
    model = KMeans(n_clusters=no_of_clusters, init='k-means++', max_iter=200, n_init=10)
    model.fit(X)

    labels=model.labels_
    clusters=pd.DataFrame(list(zip(X,labels)),columns=['title','cluster'])

    for i in range(no_of_clusters):
        print(clusters[clusters['cluster'] == i])
        
    return clusters

In [ ]:
# remove nan values from source and target domains
s = tr1.processed_text.fillna(" ")
t = tr2.processed_text.fillna(" ")
print(True in np.array(t.isna()))
print(True in np.array(s.isna()))

In [ ]:
vectorizer = TfidfVectorizer(max_features = 100) 
vectorizer.fit(s) 
source_embeddings = vectorizer.transform(s)
vectorizer.fit(t) 
target_embeddings = vectorizer.transform(t)

In [ ]:
c = cluster_text(source_embeddings)

In [ ]:
# to store the source domain clusters for future usage
c.to_csv(r'/content/drive/MyDrive/Per_CD_RS/tripcluster.csv') 

In [ ]:
# source domain cluster Dataset read from earlier stage
path = r'/content/drive/MyDrive/Per_CD_RS/tripcluster.csv'
with open(path, errors='ignore') as infile:
  d = pd.read_csv(infile)
#d.title = source_embeddings

In [ ]:
c.cluster.value_counts()

In [ ]:
set_source = []
no_clusters = len(c.cluster.value_counts())
for k in range(0, no_clusters):
  set_source.append(np.array(c[c['cluster'] == k].title))

In [ ]:
# similarity coeff v1: using temperature parameter
li = []
temp = 100
li_total = []
counter = 0
for i in target_embeddings:
  if counter % 200 == 0:
    print(counter)
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val1 = math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp)
      sim.append(val1)
    denom = sum(sim)  
    for k in sim: 
      val2 = -1 * math.log(k) / denom
      li.append(val2)
    li_total.append(sum(li) / len(li))
    li = []
  counter = counter + 1

In [ ]:
# similarity coeff v2: using normalised version with temperature parameter
li = []
denom = []
temp = 0.2
li_total = []
for i in target_embeddings:
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val = math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp)
      sim.append(val)
    denom.append(sum(sim))
  for j in range(len(set_source)):
    for k in set_source[j]:
      val = -1 * math.log(math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp) / denom[j])
      li.append(val)
    li_total.append(sum(li) / len(li))
    li = []

In [ ]:
# similarity coeff v3: cosine similarity
li = []
li_total = []
counter = 0
for i in target_embeddings:
  if counter % 200 == 0:
    print(counter)
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val1 = 1 - spatial.distance.cosine(i.toarray().flatten(), k.toarray().flatten())
      sim.append(val1)
    li_total.append(sum(sim) / len(sim))
    li = []
  counter = counter + 1

In [ ]:
li_all = np.reshape(li_total, (target_embeddings.shape[0], int(len(li_total)/target_embeddings.shape[0])))

In [ ]:
index_of_cluster = []
for i in li_all:
  i = list(i) 
  index = i.index(min(i))
  index_of_cluster.append(index)

In [ ]:
source_clusters = c
target_clusters = pd.DataFrame(list(zip(target_embeddings,index_of_cluster, tr2.rating, tr2.userId)),columns=['title','cluster', 'rating', 'userId'])

In [ ]:
source_clusters['text'] = source_texts
target_clusters['text'] = target_texts

In [ ]:
target_clusters.cluster.value_counts()

In [ ]:
# store target domain clusters for future usage
target_clusters.to_csv(r'/content/drive/MyDrive/Per_CD_RS/magz_clusters1.csv') 

In [ ]:
# target domain cluster Dataset read from earlier stage
path = r'/content/drive/MyDrive/Per_CD_RS/magz_clusters1.csv'
with open(path, errors='ignore') as infile:
  target_clusters= pd.read_csv(infile)

# BERT

In [ ]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
from transformers import logging
logging.set_verbosity_error()

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True

In [ ]:
pers_s = np.array(tr1[['open', 'cons', 'extra', 'agree', 'neuro']])
source_clusters['userId'] = np.array(tr1['userId'])
new_s = source_clusters.merge(tr1[['userId', 'open', 'cons', 'extra', 'agree', 'neuro']])

In [ ]:
# prediction of personality scores using bert model

target_output = pd.DataFrame()
target = pd.DataFrame()
traits = ["open", "cons", "extra", "agree", "neuro"]

for i in range(0, no_clusters):
  t = pd.DataFrame()
  t['userId'] = z['userId'][lx:len(z)]
  t['open'] = 0.0
  t['cons'] = 0.0
  t['extra'] = 0.0
  t['agree'] = 0.0
  t['neuro'] = 0.0
  for tr in traits:
    x = new_s[new_s['cluster']==i]
    y = target_clusters[target_clusters['cluster']==i]
    z = pd.concat([x,y])
    lx = len(x)
    ly = len(y)
    s = tr
    z = z.fillna(0)
    for j in range(len(s)):
      try:
        sample = z[['text', s[j]]]
        a = sample[0:lx]
        b = sample[lx:len(sample)]
        if len(b) > 0:
          model = ClassificationModel("bert", "bert-base", num_labels=1, args=model_args, use_cuda=False)
          print("1")
          model.train_model(a)
          print("2")
          result, model_outputs, wrong_predictions = model.eval_model(b)
          print("3")
          print(result, model_outputs, wrong_predictions)
          print("4")
          t[s[j]] = model_outputs
        else:
          print(i, j, "b empty")
      except:
            print("error")
    target_output = pd.concat([target_output, t])
    target[tr] = target_output

In [ ]:
#neuro = target_output 

In [ ]:
#target['open'] = open.open
#target['cons'] = cons.cons
#target['extra'] = extra.extra
#target['agree'] = agree.agree
#target['neuro'] = neuro.neuro
#target['userId'] = cons.userId

In [ ]:
# store the personality scores of target domain users
target.to_csv(r'/content/drive/MyDrive/Per_CD_RS/magazine_output_final.csv') 

# Data Combine

In [ ]:
# combining the personality scores across all clusters

In [ ]:
path1 = r'/content/drive/MyDrive/Per_CD_RS/magazine_output_final.csv' 

with open(path1, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile)
#arr = np.array(df[['0', '1', '2', '3', '4']])

# amazon review dataset movies json
path2 = r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv'
with open(path2, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile, nrows=100)

In [ ]:
df

In [ ]:
az

In [ ]:
t = pd.merge(df, az, on='userId', how='inner')

In [ ]:
t